# Search AiiDA Database for Slab Models

In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)
    
from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.calculation.work import WorkCalculation
from aiida.orm.calculation.job import JobCalculation

import ase.io
from base64 import b64encode
import StringIO
import numpy as np
import ipywidgets as ipw
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

from tempfile import NamedTemporaryFile

In [ ]:
############################   START OF PREPROCESSING   ###############################

In [ ]:
PREPROCESS_VERSION = 0.51

def preprocess_newbies():
    qb = QueryBuilder()
    qb.append(WorkCalculation, filters={
        'attributes._process_label': 'SlabGeoOptWorkChain',
        'or':[
               {'extras': {'!has_key': 'preprocess_version'}},
               {'extras.preprocess_version': {'<': PREPROCESS_VERSION}},
           ],
    })
    
    
    for m in qb.all(): # iterall() would interfere with set_extra()
        n = m[0]
        if not n.is_sealed:
            print("Skipping underway workchain PK %d"%n.pk)
            continue
        try:
            preprocess_one(n)
            n.set_extra('preprocess_successful', True)
            n.set_extra('preprocess_version', PREPROCESS_VERSION)
            print("Preprocessed PK %d"%n.pk)
        except Exception as e:
            n.set_extra('preprocess_successful', False)
            n.set_extra('preprocess_error', str(e))
            n.set_extra('preprocess_version', PREPROCESS_VERSION)
            print("Failed to preprocess PK %d: %s"%(n.pk, e))

In [ ]:
def preprocess_one(workcalc):
   
    def get_calc_by_label(workcalc, label):
        qb = QueryBuilder()
        qb.append(WorkCalculation, filters={'uuid':workcalc.uuid})
        qb.append(JobCalculation, output_of=WorkCalculation, filters={'label':label})
        if qb.count() != 1:
            raise(Exception("Could not find %s calculation."%label))
        calc = qb.first()[0]
        return calc

    # formula
    structure = workcalc.inp.structure
    ase_struct = structure.get_ase()
    first_slab_atom = np.argwhere(ase_struct.numbers == 79)[0, 0]
    last_slab_atom = len(ase_struct.numbers)
    mol_formula = ase_struct[:first_slab_atom].get_chemical_formula()
    slab_formula = ase_struct[first_slab_atom:].get_chemical_formula()
    workcalc.set_extra('formula', '{}_slab({})'.format(mol_formula, slab_formula))
    workcalc.set_extra('structure_description', structure.description)
    
    # optimized structure
    geopt_calc = get_calc_by_label(workcalc, "slab_geo_opt") # TODO deal with restarts, check final state
    opt_structure = geopt_calc.out.output_structure
    workcalc.set_extra('opt_structure_uuid', geopt_calc.out.output_structure.uuid)
    workcalc.set_extra('energy', geopt_calc.res.energy)

    # thumbnail
    thumbnail = render_thumbnail(ase_struct)
    workcalc.set_extra('thumbnail', thumbnail)
    
    
#     # ensure all steps succeed
#     all_steps = ['cell_opt1', 'cell_opt2', 'scf', 'export_hartree', 'bands', 'export_pdos', 'bands_lowres', 'export_orbitals']
#     if any([k.name[-1].isdigit() for k in structure.kinds]): # magnetization ?
#         all_steps.append('export_spinden')
#     for label in all_steps:
#         calc = get_calc_by_label(workcalc, label)
#         if calc.get_state() != 'FINISHED':
#             raise(Exception("Calculation %s in state %s."%(label, calc.get_state())))
#         if "aiida.out" not in calc.out.retrieved.get_folder_list():
#             raise(Exception("Calculation %s did not retrive aiida.out"%label))
#         fn = calc.out.retrieved.get_abs_path("aiida.out")
#         content = open(fn).read()
#         if "JOB DONE." not in content:
#             raise(Exception("Calculation %s did not print JOB DONE."%label))

In [ ]:
def render_thumbnail(atoms):
    tmp = NamedTemporaryFile()
    ase.io.write(tmp.name, atoms, format='png') # does not accept StringIO
    raw = open(tmp.name).read()
    tmp.close()
    return b64encode(raw)

In [ ]:
############################   END OF PREPROCESSING   ###############################

In [ ]:
def search():

    results.value = "preprocessing..."
    preprocess_newbies()
    
    results.value = "searching..."
    
    # html table header
    html  = '<style>#aiida_results td,th {padding: 2px}</style>' 
    html += '<table border=1 id="aiida_results" style="margin:10px;"><tr>'
    html += '<th>PK</th>'
    html += '<th>Creation Time</th>'
    html += '<th>Formula</th>'
    html += '<th>Energy</th>'
    html += '<th>Structure</th>'
    html += '</tr>'

    # query AiiDA database
    filters = {}
    filters['attributes._process_label'] = 'SlabGeoOptWorkChain'
    filters['extras.preprocess_version'] = PREPROCESS_VERSION
    filters['extras.preprocess_successful'] = True
    
    pk_list = inp_pks.value.strip().split()
    if pk_list:
        filters['id'] = {'in': pk_list}
        
    formula_list = inp_formula.value.strip().split()
    if inp_formula.value:
        filters['extras.formula'] = {'in': formula_list}

    qb = QueryBuilder()        
    qb.append(WorkCalculation, filters=filters)
    qb.order_by({WorkCalculation:{'ctime':'desc'}})

    for i, node_tuple in enumerate(qb.iterall()):
        node = node_tuple[0]
        thumbnail = node.get_extra('thumbnail')
        description = node.get_extra('structure_description')
        opt_structure_uuid = node.get_extra('opt_structure_uuid')
        
        # append table row
        html += '<tr>'
        html += '<td>%d</td>' % node.pk
        html += '<td>%s</td>' % node.ctime.strftime("%Y-%m-%d %H:%M")
        html += '<td>%s</td>' % node.get_extra('formula')
        html += '<td>%f</td>' % node.get_extra('energy')
        html += '<td><a target="_blank" href="../export_structure.ipynb?uuid=%s">'%opt_structure_uuid
        html += '<img width="100px" src="data:image/png;base64,%s" title="%s"></a></td>' % (thumbnail, description)
        html += '</td>'
        html += '</tr>'

    html += '</table>'
    html += 'Found %d matching entries.<br>'%qb.count()

    results.value = html

In [ ]:
# search UI
style = {"description_width":"100px"}
layout = ipw.Layout(width="592px")
inp_pks = ipw.Text(description='PKs', placeholder='e.g. 4062 4753 (space separated)', layout=layout, style=style)
inp_formula = ipw.Text(description='Formulas:', placeholder='e.g. C44H16 C36H4', layout=layout, style=style)

search_crit = [inp_pks, inp_formula]

In [ ]:
def on_click(b):
    with info_out:
        clear_output()
        search()

button = ipw.Button(description="Search")
button.on_click(on_click)
results = ipw.HTML()
info_out = ipw.Output()
app = ipw.VBox(children=search_crit + [button, results, info_out])
display(app)